In [2]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [3]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [4]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 30
n_iter = 13000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [5]:
import impl.RNN as nn
net = nn.RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.LSTM as nn
# net = nn.LSTM(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU as nn
# net = nn.GRU(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU2 as nn
# net = nn.GRU2(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU3 as nn
# net = nn.GRU3(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# # GRU4 goes to inf because the res-conn is gone.
# import impl.GRU4 as nn
# net = nn.GRU4(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [6]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step, # This is the width of the network
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.2999
hashitke riulatbin oiOnlacGs fSineadmCid 'i deitic ahand ireaiivk, irts7 ss 1alegso anthe hicaihinf 


Iter-2000 loss: 2.5428
tof Japenesos of ic rc4tar us the taoniss eed anelares lsrinit of uroond os upery bonl ry tce d cend


Iter-3000 loss: 2.0611
warly fImina dile e2t es ind iso1yd in 1r'e pongecea.e sy eIjden-latex Toizd wares louhecoroli. Thic


Iter-4000 loss: 1.7583
stacin k7d werciry sid thith ctwisnd wingt urg th mamboustorth ithecd-rupet in sSen GPealld Nito og 


Iter-5000 loss: 1.5396
 ano unt
 "201stican cLCeIncya's Sins iratalon trkand 2riy. T8echukelgy tie parghitn iny 85-rydbt wo


Iter-6000 loss: 1.3877
0st anero-tar wos ocede Gns oEunthe countivizalithiis anored intlryssrlagissogestity 1stinularee pre


Iter-7000 loss: 1.2646
d Japan was anoese pal exper1bar ofed itte as ef Japandis d ofulitala untc as lorode is jrg rt thich


Iter-8000 loss: 1.1676
 divile pomes anm oxet oris. .n6 Nicite the the Okto-lergh is riost. "ter. Japan-cane Wisgost i

In [ ]:
=========================================================================
Iter-299000 loss: 0.0278
=========================================================================
s. Japan is a developed country with a high standard of living and Human Development Index whose pop
=========================================================================


=========================================================================
Iter-300000 loss: 0.0268
=========================================================================
apan to open to the West. Nearly two decades of internal conthic-komy parionory. From the 12th centu
=========================================================================


Out[5]:
<impl.GRU.GRU at 0x7f1401bc5588>

In [ ]:
=========================================================================
Iter-299000 loss: 0.0914
=========================================================================
s. Japan's history. From the 12th century until 1868, Japan's n7meamean Laas pan hs, Japan was men M
=========================================================================


=========================================================================
Iter-300000 loss: 0.1100
=========================================================================
apan was inhabited as early as the Upper Paleolithic period. The first are Honsh mict of inter of in
=========================================================================


Out[10]:
<impl.LSTM.LSTM at 0x7f51924f1940>

In [ ]:
=========================================================================
Iter-299000 loss: 0.1178
=========================================================================
s. Japan is a developed country wino avinsm itso. The foulslint of is in the namiort tites Toky byde
=========================================================================


=========================================================================
Iter-300000 loss: 0.0913
=========================================================================
apan was ruled in the name of the Emperor as a d bevere Hf Hiros. Japan is athe world's third-s. Jap
=========================================================================


Out[8]:
<impl.RNN.RNN at 0x7f51924f1630>